In [1]:
import torch

In [2]:
%cd '/home/ext_yao_gary_mayo_edu/CLAM'

/home/ext_yao_gary_mayo_edu/CLAM


In [24]:
from models.model_clam import CLAM_MB
import torch
import os
import h5py
from torch.autograd import grad
import openslide
from datasets.dataset_h5 import eval_transforms
from models.resnet_custom import resnet50_baseline
from torchvision import transforms, utils, models
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [4]:
model = torch.load('/home/jupyter/ash_saves/ash_epoch_res19.save').cpu()

In [5]:
resnet = resnet50_baseline()

In [6]:
current_features_path = '/home/ext_yao_gary_mayo_edu/post_process/duodenum_2/20x_features/h5_files/10515437_XR17-663 B1-1_1-9-2017_HE.h5'

In [7]:
with h5py.File(current_features_path, 'r') as hdf5_file:
                features = hdf5_file['features'][:]
                coords = hdf5_file['coords'][:]

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

toTensor = transforms.ToTensor()
normalize = transforms.Normalize(mean = mean, std = std)

In [66]:
def explain(resnet, model, slide_name, target_idx, features_dir, slide_dir, target_label = 0, full_pipeline = False):
    #load features
    target_features_path = os.path.join(features_dir,
                                '{}.h5'.format(slide_name))
    with h5py.File(target_features_path, 'r') as hdf5_file:
                features = hdf5_file['features'][:]
                coords = hdf5_file['coords'][:]
    features = torch.from_numpy(features).cuda()
    
    #grab targeted tile from wsi
    target_coords = coords[target_idx]
    target_coords_x, target_coords_y = target_coords
    print(target_coords_x, target_coords_y)
    target_slide_path = os.path.join(slide_dir,'{}.svs'.format(slide_name))
    wsi = openslide.open_slide(target_slide_path)
    target_tile = wsi.read_region((target_coords_x, target_coords_y), 0, (512,512)).convert('RGB')
    #display(target_tile)
    target_tile = toTensor(target_tile)
    target_tile = normalize(target_tile).unsqueeze(0)
    #pass image through resnet
    
    y = resnet(target_tile)
    
    #replace target feature and pass through model, if needed
    if not full_pipeline:
        l,y,y1,a,r  = model(y)
        print(l.shape)
    
    #back prop and grab target gradient
    loss_metric = torch.nn.CrossEntropyLoss()
    
    target_label = torch.Tensor([target_label]).long()
    loss = loss_metric(y, target_label)
    loss.backward()
    
    gradients = resnet.grad
    #print(grad.shape)
    
    activations = resnet.get_activation(target_tile).detach()
    #print(activation.shape)
    #generate explaination heatmap
    

    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])


    for i in range(512):
        activations[:, i, :, :] *= pooled_gradients[i]

    heatmap = torch.mean(activations, dim=1).squeeze()

    heatmap = np.maximum(heatmap, 0)

    heatmap /= torch.max(heatmap)

    heatmap.squeeze()
    img = wsi.read_region((target_coords_x, target_coords_y), 0, (512,512)).convert('RGB')
    img = np.array(img)
    print(img.shape)
    heatmap = cv2.resize(np.float32(heatmap), (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = .4 * heatmap + .6 * img
    cv2.imwrite('/home/jupyter/map.jpg', superimposed_img)

#### slide_name = 'XR12-27403_A'
target_idx = 7
slides_dir = '/home/ext_yao_gary_mayo_edu/datasets/duodenum/full/'
features_dir = '/home/ext_yao_gary_mayo_edu/post_process/duodenum_2/20x_features/h5_files/'

explain(resnet, model, slide_name, target_idx, features_dir, slides_dir, target_label = 0)